## **Robot Model** 

`RobotModel` is a tiny wrapper around the modeling backend (`casadi_kin_dyn`) that allow you to create symbolical CasAdi model from **urdf** file, then calculate a notions such as **forward and inverse dynamics**, com positions, state space representation etc, one may also calculate **body** specific properties and add **contacts**.

The typical workflow as follows:
* Build the urdf of your robot and create `RobotModel` instance
* Add additional bodies and possibly contacts
* Calculate the all necessary functions with `~.update_model()` method 
* Access to casadi functions that are stored within RobotModel and use them either numerically or symbolically in other CasAdi empowered projects.  

There are also banch of modules that facilitates work with given type of robots, i.e. manipulators, quadrupeds and bipeds.


As example let us create the model of z1 manipulator:

We will use [robot_descriptions.py](https://github.com/robot-descriptions/robot_descriptions.py) library to simplify the process of urdf retrieval.

### *Note*:
You have to restart kernel after `robot_descriptions` is installed

In [1]:
%%capture
!pip3 install robot_descriptions

In [2]:
from darli.robots import RobotModel
from robot_descriptions import z1_description

model = RobotModel(z1_description.URDF_PATH)

One can retrive basic model info as follows:

In [3]:
nq = model.nq # dimensionality of configuration  
nv = model.nv # dimensionality of generilized velocities
nu = model.nu # dimensionality  of control inputs
q_min, q_max = model.q_min, model.q_max # minimal and maximal limits on joint pos
nq, nv, nu

(6, 6, 6)

In [4]:
joint_names = model.joint_names # names of the joints
joint_names

['joint1', 'joint2', 'joint3', 'joint4', 'joint5', 'joint6']

In [5]:
model.joint_iq(joint_names[nv-1]) # will return id of the joint by name

5

### **Equations of Motion and Dynamics**

The dynamics of articulated mechanics in robotic systems is usually represented as:
$$
\mathbf{M}(\mathbf{q})\dot{\mathbf{v}} + \mathbf{C}(\mathbf{q},\mathbf{v})\mathbf{v} + \mathbf{g}(\mathbf{q})  = 
\mathbf{M}(\mathbf{q})\dot{\mathbf{v}} + \mathbf{c}(\mathbf{q},\mathbf{v}) + \mathbf{g}(\mathbf{q}) = \mathbf{M}(\mathbf{q})\dot{\mathbf{v}} + \mathbf{h}(\mathbf{q},\mathbf{v}) = \mathbf{Q}
$$

where:
* $\mathbf{Q} \in \mathbb{R}^{nv}$ - generalized forces corresponding to generilized coordinates
* $\mathbf{q} \in \mathbb{R}^{nq}$ - vector of generilized coordinates
* $\mathbf{v} \in \mathbb{R}^{nq}$ - vector of generilized velocities (sometimes $\mathbf{v} = \dot{\mathbf{q}}$, but not in case of $\mathbf{q}$ containing quaternions)
* $\mathbf{M} \in \mathbb{R}^{nv \times nv}$ - positive definite symmetric inertia matrix 
* $\mathbf{c} \in \mathbb{R}^{nv}$ - describe centrifugal and Coriolis forces
* $\mathbf{C} \in \mathbb{R}^{nv \times nv}$ - describe 'coefficients' of centrifugal and Coriolis forces
* $\mathbf{g} \in \mathbb{R}^{nv}$ - describes effect of gravity and other position depending forces
* $\mathbf{h} \in \mathbb{R}^{nv} $ - combined effect of $\mathbf{g}$ and $\mathbf{c}$


One can get all of the above quantities in symbotics as follows:

In [6]:
inertia = model.inertia
gravity_vector = model.gravity
bias_force = model.bias_force
coriolis_matrix = model.coriolis_matrix 
coriolis = model.coriolis

Each of the above define the CasAdi functions:

In [7]:
inertia

Function(inertia:(q[6])->(inertia[6x6]) SXFunction)

And can be evaluated both numerically and symbolically:

In [8]:
import numpy as np 
import casadi as cs

# Symbolical computation
print('Symbolical:', inertia(cs.SX.sym('q', nq)))
# Numerical computation
print('Numerical:', inertia(np.random.randn(nq)))

Symbolical: @1=7e-07, @2=sin(q_5), @3=-1.6e-07, @4=cos(q_5), @5=((@1*@2)+(@3*@4)), @6=(@5*@2), @7=0.0001468, @8=0.165803, @9=0.00177591, @10=-0.00017355, @11=-0.00143876, @12=((@10*@4)-(@11*@2)), @13=((0.0005386+(@6+@7))+(@8*(@9+sq(@12)))), @14=0.307927, @15=0.07, @16=0.0491595, @17=cos(q_4), @18=sin(q_4), @19=0.425807, @20=(@19*@12), @21=(@15+((@16*@17)-(@18*@20))), @22=(0.0436668-@21), @23=((@16*@18)+(@17*@20)), @24=(0.00364738-@23), @25=((0.00097912+@13)+(@14*(sq(@22)+sq(@24)))), @26=cos(q_3), @27=2, @28=-1.22e-06, @29=-1.08e-06, @30=-5.4e-07, @31=-0.00698719, @32=((@10*@2)+(@11*@4)), @33=(0.00646316-@32), @34=((-5.689e-05+(((@28*@2)+(@29*@4))-(@30*@4)))-(@31*@33)), @35=(@27*@34), @36=((1.3e-07+((@5*@4)+8e-08))+((@8*@12)*@33)), @37=(@36+@36), @38=(0.0037111+(@19*@32)), @39=(-0.00170192-@38), @40=((-4.091e-05+(((@35*@17)-(@37*@18))-((@17*@34)-(@18*@36))))-((@14*@22)*@39)), @41=sin(q_3), @42=(((@1*@4)-(@3*@2))*@4), @43=(((0.00017605+(((3.718e-05-@42)-@6)+@7))+(@8*(sq(@12)+sq(@33))))-@

Note that above are calculated not via Lagrange formalism but by using efficient recursive Newton-Euler algorithm (for `bias_force`), while inertia matrix is evaluated by composite rigid body algorithm (CRBA)

There are some notions apart from dynamical coefficients that may be useful as well, such as com kinematics and jacobians, and energy:

In [9]:
com_position = model.com().position
com_velocity = model.com().velocity
com_jacobian = model.com().jacobian
com_jacobian_dt = model.com().jacobian_dt
potential_energy = model.energy().potential
kinetic_energy = model.energy().kinetic
lagrangian = model.lagrangian

# TODO
# momentum

In [10]:
com_jacobian_dt

Function(com_jacobian_dt:(q[6],v[6])->(com_jacobian_dt[3x6]) SXFunction)

#### **Forward and Inverse Dynamics**


In inverse dynamics we looking for the generilized forces:
$$
    \mathbf{Q} = \mathbf{M}(\mathbf{q})\dot{\mathbf{v}} + \mathbf{h}(\mathbf{q},\mathbf{v}) = \text{rnea}(\mathbf{q}, \mathbf{v}, \dot{\mathbf{v}})
$$

In [11]:
model.inverse_dynamics

Function(inverse_dynamics:(q[6],v[6],dv[6])->(tau[6]) SXFunction)

While forward dynamics is basically solving the equations of motion with respect to accelerations $\dot{\mathbf{v}}$. However the solution of the above in general case usually yields the complexity of $O(nv^3)$, for this reason a way to go is to use celebrated Featherstone Articulated Body algorithm (ABA), which effectevely exploit sparsity of $\mathbf{M}$ for the tree structures and produce nearly linear complexity $O(nv)$:


$$
\dot{\mathbf{v}} = \mathbf{M}^{-1}(\mathbf{q})(\mathbf{Q} - \mathbf{h}(\mathbf{q},\mathbf{v})) = \text{aba}(\mathbf{q}, \mathbf{v}, \mathbf{Q})
$$

<!-- Articulated body algorithm 

Feather stone algorithm
 -->
<!-- http://gamma.cs.unc.edu/courses/robotics-f08/LEC/ArticulatedBodyDynamics.pdf -->

In [12]:
model.forward_dynamics

Function(forward_dynamics:(q[6],v[6],u[6])->(dv[6]) SXFunction)

#### **Passive Joints and Selector**

In practical situations 

Choosing the passive joints:

In [13]:
import numpy as np 
print(f'Old input dimensions: {model.nu}')
S = np.random.randn(model.nv, model.nv+2)
model.set_selector(S)
print(f'New input dimensions: {model.nu}')

Old input dimensions: 6
New input dimensions: 8


In [14]:
model.forward_dynamics

Function(forward_dynamics:(q[6],v[6],u[8])->(dv[6]) SXFunction)

In [15]:
model.set_selector(passive_joints=range(3))
print(f'New input dimensions: {model.nu}')
print(f'New selector:\n {model.selector}')

New input dimensions: 3
New selector:
 [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [16]:
model.forward_dynamics

Function(forward_dynamics:(q[6],v[6],u[3])->(dv[6]) SXFunction)

#### **State space Representation and Linearization**

One can easily transform the mechanical system to the state space form by defining the state $\mathbf{x} = [\mathbf{q}, \mathbf{v}]^T$:


$$
\dot{\mathbf{x}}= \mathbf{f}(\mathbf{x}, \mathbf{u}) = 
\begin{bmatrix}
\dot{\mathbf{x}}_1 \\ 
\dot{\mathbf{x}}_2
\end{bmatrix}=
\begin{bmatrix}
\dot{\mathbf{q}} \\ 
\dot{\mathbf{v}}
\end{bmatrix}=
\begin{bmatrix}
\mathbf{W}(\mathbf{q})\mathbf{v} \\ 
\text{aba}(\mathbf{q}, \mathbf{v}, \mathbf{S}\mathbf{u})
\end{bmatrix}=
\begin{bmatrix}
\mathbf{W}(\mathbf{x}_1)\mathbf{x}_2 \\
\text{aba}(\mathbf{x}_1, \mathbf{x}_2, \mathbf{S}\mathbf{u})
\end{bmatrix}
$$



In [17]:
model.state_space

The above equation can be easily linearized to produce following linear approximation:

One can easily find linearization with respect to state:

In [18]:
model.state_space.state_jacobian

Function(state_jacobian:(x[12],u[3])->(dfdx[12x12,72nz]) SXFunction)

and control:

In [19]:
model.state_space.input_jacobian

Function(input_jacobian:(x[12],u[3])->(dfdu[12x3,18nz]) SXFunction)

These functionality allows for easy implementation of linearization based analysis and control.

#### **Bodies and Contacts**

In [20]:
model.add_bodies(['link06'])
model.bodies_names

{'link06'}

In [21]:
model.body('link06')

One may also retrieve a hash map of all bodies:

In [22]:
model.bodies

{'link06': <darli.models.body.Body at 0x7f1dcd5e64d0>}

In [23]:
model.body("link06").linear_acceleration().local

Function(link06_linear_acceleration_local:(q[6],v[6],dv[6])->(linear_acceleration_local[3]) SXFunction)

In [24]:
model.body("link06").jacobian().local

Function(link06_jacobian_local:(q[6])->(jacobian[6x6]) SXFunction)

In [25]:
model.body("link06").jacobian().local
model.body("link06").jacobian_dt().local
model.body("link06").linear_velocity().local
model.body("link06").angular_velocity().local
model.body("link06").linear_acceleration().local
model.body("link06").angular_acceleration().local

Function(link06_angular_acceleration_local:(q[6],v[6],dv[6])->(angular_acceleration_local[3]) SXFunction)

The body jacobian and velocities can be calculated with respect to `world`, `local` and `world_aligned` frames.

In [26]:
model.body("link06").jacobian().world

Function(link06_jacobian_world:(q[6])->(jacobian[6x6]) SXFunction)

Note that body name can be initialized with dictionary that maps given name to one presented in urdf i.e: `{'ee':'link06'}`

##### **Contacts**

In [27]:
model.body("link06").add_contact("wrench", "world")

In [28]:
model.body("link06").contact.dim
model.body("link06").contact.contact_frame
model.body("link06").contact.reference_frame
model.body("link06").contact.contact_qforce

Function(link06_qforce_world:(q[6],wrench_force[6])->(generilized_force[6]) SXFunction)

Do not forget to rebuild the model:

In [29]:
model.update_model()

Note how arguments are changed in dynamics related functions, i.e:

In [30]:
model.forward_dynamics

Function(forward_dynamics:(q[6],v[6],u[3],link06[6])->(dv[6]) SXFunction)

the state space representation and jacobians are changed as well:

In [31]:
model.state_space.state_derivative

Function(state_derivative:(x[12],u[3],link06[6])->(dxdt[12]) SXFunction)

In [32]:
model.state_space.state_jacobian

Function(state_jacobian:(x[12],u[3],link06[6])->(dfdx[12x12,78nz]) SXFunction)

In [33]:
model.body("link06").contact.add_cone(mu=0.5, X=0.05, Y=0.02)

In [34]:
wrench_cone = model.body("link06").contact.cone.full()

In [35]:
model.body("link06").contact.cone.linearized()

DM(
[[-1, 0, -0.5, 0, 0, 0], 
 [1, 0, -0.5, 0, 0, 0], 
 [0, -1, -0.5, 0, 0, 0], 
 [0, 1, -0.5, 0, 0, 0], 
 [0, 0, -0.02, -1, 0, 0], 
 [0, 0, -0.02, 1, 0, 0], 
 [0, 0, -0.05, 0, -1, 0], 
 [0, 0, -0.05, 0, 1, 0], 
 [-0.02, -0.05, -0.035, 0.5, 0.5, -1], 
 [-0.02, 0.05, -0.035, 0.5, -0.5, -1], 
 [0.02, -0.05, -0.035, -0.5, 0.5, -1], 
 [0.02, 0.05, -0.035, -0.5, -0.5, -1], 
 [0.02, 0.05, -0.035, 0.5, 0.5, 1], 
 [0.02, -0.05, -0.035, 0.5, -0.5, 1], 
 [-0.02, 0.05, -0.035, -0.5, 0.5, 1], 
 [-0.02, -0.05, -0.035, -0.5, -0.5, 1]])

In [36]:
model.add_bodies({"ee": "link06"})
model.bodies_names

{'ee': 'link06'}

One can add bodies on the initialization stage based on following syntax:

In [37]:
RobotModel(z1_description.URDF_PATH, bodies_names={'shoulder':'link03', 'ee':'link06'})

The `bodies_names` arguments can be listof body names present in urdf, however for increased readability we suggest to use the dictionary as shown above.

## **Prespecified Robots**

One can build different robots....

In [38]:
from darli.robots import Biped, Manipulator, Quadruped

As example let us consider the Atlas humanoid robot:

Note: robot loaded in example is `fixed` in its pelvis and in real world, you have to create a floating base model to have a full set of generalized coordinates.

In [39]:
from robot_descriptions import atlas_v4_description

biped_urdf = atlas_v4_description.URDF_PATH

biped_model = Biped(
    biped_urdf,
    torso={"torso": "pelvis"},
    foots={"left_foot": "l_foot", "right_foot": "r_foot"},
)

In [40]:
biped_model.forward_dynamics

Function(forward_dynamics:(q[30],v[30],u[24],left_foot[6],right_foot[6])->(dv[30]) SXFunction)

In [41]:
biped_model.inverse_dynamics

Function(inverse_dynamics:(q[30],v[30],dv[30],left_foot[6],right_foot[6])->(tau[30]) SXFunction)

In [42]:
biped_model.body("torso").position

Function(position_torso:(q[30])->(position[3]) SXFunction)

In [43]:
biped_model.body("torso").rotation

Function(rotation_torso:(q[30])->(rotation[3x3]) SXFunction)

In [44]:
biped_model.body("torso").jacobian().world_aligned

Function(torso_jacobian_world_aligned:(q[30])->(jacobian[6x30]) SXFunction)

In [45]:
biped_model.body("torso").angular_velocity().local

Function(torso_angular_velocity_local:(q[30],v[30])->(angular_velocity_local[3]) SXFunction)

In [46]:
biped_model.body("torso").jacobian_dt().world_aligned

Function(torso_djacobian_world_aligned:(q[30],v[30])->(djacobian[6x30]) SXFunction)

In [47]:
biped_model.body("left_foot").contact.jacobian()

{'contact_jacobian': DM(
 [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -0.0225, -0, -0.796, -0.422, 0, 0, 0, 0, 0, 0, 0, 0], 
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.862, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0225, 0.05, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, -0, -0, -0, 1, 0, 0, 0, 0, 0, 0], 
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0], 
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, -0, 0, 0, 0, 0, 0, 0]])}

In [48]:
biped_model.body("right_foot").contact.jacobian()

{'contact_jacobian': DM(
 [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0225, -0, -0.796, -0.422, 0, 0], 
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.862, 0, 0, 0, 0], 
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -0.0225, 0.05, 0, 0, 0], 
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, -0, -0, -0, 1], 
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0], 
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, -0]])}

One can create a structure for specific robot, the typical reciept as follows:
* Create the child class with parent being `RobotModel`.
* Add bodies bodies and necessary contacts
* Calculate the all necessary functions with `~.update_model()` method 

An example that creates Quadruped robot possibly with arm:

In [49]:
class Quadruped(RobotModel):
    def __init__(
        self,
        urdf_path,
        torso=None,
        foots=None,
        arm=None,
        reference="world_aligned",
        calculate=True,
    ):
        bodies_names = {}

        if torso is not None:
            if isinstance(torso, str):
                bodies_names.update([torso])
            else:
                bodies_names.update(torso)

        if arm is not None:
            if isinstance(arm, str):
                bodies_names.update([arm])
            else:
                bodies_names.update(arm)

        if foots is not None:
            bodies_names.update(foots)

        super().__init__(urdf_path, bodies_names=bodies_names, calculate=False)

        for foot in foots.keys():
            body = self.body(foot)
            body.add_contact(frame=reference, contact_type="point")

        self.set_selector(passive_joints=range(6), calculate=False)
        self.update_model()